In [17]:
from numpy import *
import numpy as np
import pandas as pd
import pylab as pl
import matplotlib.pyplot as plt
from numpy import linalg as la
import pdb
%matplotlib inline

In [60]:
rr = pd.read_json('renttherunway_final_data.json',lines=True)
print(shape(rr))
rr.head(5)

category_list = ['gown','jacket']

rr = rr[rr['category'].isin(category_list)]

(192544, 15)


In [61]:
rr.head()

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
1,fit,273551,34b,153475,132lbs,10.0,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,"5' 6""",12,36.0,"June 18, 2013"
4,fit,151944,34b,616682,145lbs,10.0,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,"5' 9""",12,27.0,"September 26, 2016"
8,fit,166228,36d,1729232,NaN,10.0,formal affair,I was nervous of it looking cheap when it arri...,full bust,Great for black tie event!,gown,"5' 6""",21,27.0,"June 27, 2016"
9,fit,154309,32b,1729232,114lbs,10.0,formal affair,The dress was very flattering and fit perfectl...,petite,This dress was everything! It was perfect for ...,gown,"5' 3""",1,33.0,"October 17, 2016"
14,fit,721308,34b,123793,118lbs,10.0,formal affair,"Fit great, super flattering",athletic,Stunning gown. Wore this for heart ball and re...,gown,"5' 5""",2,32.0,"May 29, 2014"


##### Number of Users Change
This MF uses 2500 users. 

In [62]:
#reorder id's
item_id = rr['item_id']
user_id = rr['user_id']
rating = rr['rating']

rr.drop('item_id', axis = 1,inplace = True)
rr.insert(0,'item_id',item_id)
rr.drop('user_id', axis = 1, inplace  = True)
rr.insert(1,'user_id',user_id)
rr.drop('rating', axis = 1,inplace = True)
rr.insert(2,'rating',rating)

n_rr = rr.iloc[:2500, :]

print(shape(n_rr))


(2500, 15)


##### Drop in Duplicate Change:
This example of matrix factorization does not drop the duplicate user or item id. 

In [72]:
# rr_Sorted = rr.sort_values(['user_id'])
# n_rr = rr_Sorted.drop_duplicates(['user_id'])
# shape(n_rr)

# n_rr_Sorted = n_rr.sort_values(['item_id'])
# n_rr = n_rr_Sorted.drop_duplicates(['item_id'])
# shape(n_rr)

n_rr_rating = n_rr.iloc[:,0:3]

rr_rating.item_id = n_rr_rating.item_id.astype(object)
rr_rating.user_id = n_rr_rating.user_id.astype(object)

##### User x Item Matrix

In [64]:
# user x item
R = n_rr_rating.pivot_table(index = ['user_id'],columns = ['item_id'],values = "rating")
R1 = R.replace(np.nan,0)
print(shape(R1))
print(R1.head())
R1 = np.array(R1)

(2477, 481)
item_id  123373   123793   125465   125564   127081   127865   128959   \
user_id                                                                  
1066         0.0      0.0      0.0     10.0      0.0      0.0      0.0   
1141        10.0      0.0      0.0      0.0      0.0      0.0      0.0   
1921         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1939         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1973         0.0      0.0      0.0      0.0      0.0      0.0      0.0   

item_id  131117   131533   131698   ...  2872180  2872227  2904743  2918587  \
user_id                             ...                                       
1066         0.0      0.0      0.0  ...      0.0      0.0      0.0      0.0   
1141         0.0      0.0      0.0  ...      0.0      0.0      0.0      0.0   
1921         0.0      0.0      0.0  ...      0.0      0.0      0.0      0.0   
1939         0.0      0.0      0.0  ...      0.0      0.0      0.0      0.

In [65]:
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    
    ### R = The user x item rating matrix (m x n)
    ### P = Initial user-factor matrix (m x k)
    ### Q = Initial item-factor matrix (n x k)
    ### K = The number of latent factors (features)
    ### steps = The number of epochs in gradient descent
    ### alpha = The learning rate for gradient descent
    ### beta = The regularization coefficient
    
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        ### update P and Q based on the partial derivatives
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
        print("Step %d of %d; Error: %0.5f; Time: %0.2f" %(step+1, steps, e, time()))
    return P, Q.T

In [67]:
M = R1.shape[0]
N = R1.shape[1]
K = 12
P = np.random.rand(M,K)
Q = np.random.rand(N,K)

from time import time
t0 = time()

fP, fQ = matrix_factorization(R1, P, Q, 12, steps=1500, alpha=0.0002, beta=0.02)
print("done in %0.3fs." % (time() - t0))


Step 1 of 1500; Error: 95737.91110; Time: 1605821908.22
Step 2 of 1500; Error: 90925.48820; Time: 1605821910.23
Step 3 of 1500; Error: 86633.78573; Time: 1605821912.11
Step 4 of 1500; Error: 82784.71420; Time: 1605821914.40
Step 5 of 1500; Error: 79313.23191; Time: 1605821916.40
Step 6 of 1500; Error: 76164.99946; Time: 1605821918.24
Step 7 of 1500; Error: 73294.52116; Time: 1605821920.45
Step 8 of 1500; Error: 70663.64363; Time: 1605821923.25
Step 9 of 1500; Error: 68240.32587; Time: 1605821925.43
Step 10 of 1500; Error: 65997.62210; Time: 1605821927.64
Step 11 of 1500; Error: 63912.83572; Time: 1605821929.81
Step 12 of 1500; Error: 61966.81359; Time: 1605821931.99
Step 13 of 1500; Error: 60143.35667; Time: 1605821934.37
Step 14 of 1500; Error: 58428.72809; Time: 1605821937.19
Step 15 of 1500; Error: 56811.24290; Time: 1605821940.07
Step 16 of 1500; Error: 55280.92657; Time: 1605821942.56
Step 17 of 1500; Error: 53829.23107; Time: 1605821945.45
Step 18 of 1500; Error: 52448.79934; Tim

Step 145 of 1500; Error: 6216.13043; Time: 1605822233.47
Step 146 of 1500; Error: 6130.29501; Time: 1605822235.40
Step 147 of 1500; Error: 6045.82758; Time: 1605822237.86
Step 148 of 1500; Error: 5962.70522; Time: 1605822239.98
Step 149 of 1500; Error: 5880.90542; Time: 1605822242.33
Step 150 of 1500; Error: 5800.40611; Time: 1605822244.39
Step 151 of 1500; Error: 5721.18562; Time: 1605822246.61
Step 152 of 1500; Error: 5643.22267; Time: 1605822248.87
Step 153 of 1500; Error: 5566.49637; Time: 1605822251.29
Step 154 of 1500; Error: 5490.98622; Time: 1605822253.56
Step 155 of 1500; Error: 5416.67207; Time: 1605822255.74
Step 156 of 1500; Error: 5343.53414; Time: 1605822258.56
Step 157 of 1500; Error: 5271.55303; Time: 1605822260.93
Step 158 of 1500; Error: 5200.70965; Time: 1605822263.85
Step 159 of 1500; Error: 5130.98526; Time: 1605822266.17
Step 160 of 1500; Error: 5062.36148; Time: 1605822268.39
Step 161 of 1500; Error: 4994.82022; Time: 1605822270.56
Step 162 of 1500; Error: 4928.3

Step 289 of 1500; Error: 1284.62365; Time: 1605822529.47
Step 290 of 1500; Error: 1275.36247; Time: 1605822531.30
Step 291 of 1500; Error: 1266.23867; Time: 1605822533.16
Step 292 of 1500; Error: 1257.25013; Time: 1605822535.39
Step 293 of 1500; Error: 1248.39480; Time: 1605822537.42
Step 294 of 1500; Error: 1239.67064; Time: 1605822539.89
Step 295 of 1500; Error: 1231.07566; Time: 1605822542.18
Step 296 of 1500; Error: 1222.60787; Time: 1605822544.11
Step 297 of 1500; Error: 1214.26534; Time: 1605822546.00
Step 298 of 1500; Error: 1206.04617; Time: 1605822547.86
Step 299 of 1500; Error: 1197.94846; Time: 1605822549.67
Step 300 of 1500; Error: 1189.97039; Time: 1605822551.79
Step 301 of 1500; Error: 1182.11012; Time: 1605822553.68
Step 302 of 1500; Error: 1174.36586; Time: 1605822555.65
Step 303 of 1500; Error: 1166.73585; Time: 1605822557.56
Step 304 of 1500; Error: 1159.21835; Time: 1605822559.39
Step 305 of 1500; Error: 1151.81166; Time: 1605822561.23
Step 306 of 1500; Error: 1144.5

Step 435 of 1500; Error: 723.91151; Time: 1605822798.97
Step 436 of 1500; Error: 722.79052; Time: 1605822800.99
Step 437 of 1500; Error: 721.68504; Time: 1605822802.69
Step 438 of 1500; Error: 720.59483; Time: 1605822804.43
Step 439 of 1500; Error: 719.51968; Time: 1605822806.42
Step 440 of 1500; Error: 718.45937; Time: 1605822808.37
Step 441 of 1500; Error: 717.41368; Time: 1605822809.93
Step 442 of 1500; Error: 716.38241; Time: 1605822811.53
Step 443 of 1500; Error: 715.36534; Time: 1605822813.22
Step 444 of 1500; Error: 714.36227; Time: 1605822815.00
Step 445 of 1500; Error: 713.37300; Time: 1605822816.74
Step 446 of 1500; Error: 712.39733; Time: 1605822818.58
Step 447 of 1500; Error: 711.43506; Time: 1605822820.44
Step 448 of 1500; Error: 710.48599; Time: 1605822822.19
Step 449 of 1500; Error: 709.54994; Time: 1605822824.04
Step 450 of 1500; Error: 708.62672; Time: 1605822826.11
Step 451 of 1500; Error: 707.71614; Time: 1605822827.81
Step 452 of 1500; Error: 706.81802; Time: 160582

Step 582 of 1500; Error: 651.54268; Time: 1605823089.33
Step 583 of 1500; Error: 651.37339; Time: 1605823091.41
Step 584 of 1500; Error: 651.20600; Time: 1605823093.36
Step 585 of 1500; Error: 651.04048; Time: 1605823095.21
Step 586 of 1500; Error: 650.87680; Time: 1605823097.28
Step 587 of 1500; Error: 650.71495; Time: 1605823099.17
Step 588 of 1500; Error: 650.55489; Time: 1605823101.07
Step 589 of 1500; Error: 650.39660; Time: 1605823102.80
Step 590 of 1500; Error: 650.24006; Time: 1605823104.69
Step 591 of 1500; Error: 650.08524; Time: 1605823106.60
Step 592 of 1500; Error: 649.93212; Time: 1605823108.43
Step 593 of 1500; Error: 649.78068; Time: 1605823110.37
Step 594 of 1500; Error: 649.63089; Time: 1605823112.18
Step 595 of 1500; Error: 649.48273; Time: 1605823113.95
Step 596 of 1500; Error: 649.33618; Time: 1605823115.48
Step 597 of 1500; Error: 649.19121; Time: 1605823117.10
Step 598 of 1500; Error: 649.04781; Time: 1605823118.65
Step 599 of 1500; Error: 648.90595; Time: 160582

...

Step 876 of 1500; Error: 633.41115; Time: 1605823675.85
Step 877 of 1500; Error: 633.38581; Time: 1605823678.16
Step 878 of 1500; Error: 633.36054; Time: 1605823680.29
Step 879 of 1500; Error: 633.33534; Time: 1605823682.66
Step 880 of 1500; Error: 633.31020; Time: 1605823685.04
Step 881 of 1500; Error: 633.28513; Time: 1605823687.20
Step 882 of 1500; Error: 633.26013; Time: 1605823689.63
Step 883 of 1500; Error: 633.23519; Time: 1605823691.98
Step 884 of 1500; Error: 633.21031; Time: 1605823694.30
Step 885 of 1500; Error: 633.18550; Time: 1605823696.65
Step 886 of 1500; Error: 633.16075; Time: 1605823699.07
Step 887 of 1500; Error: 633.13607; Time: 1605823701.29
Step 888 of 1500; Error: 633.11144; Time: 1605823703.65
Step 889 of 1500; Error: 633.08688; Time: 1605823706.03
Step 890 of 1500; Error: 633.06238; Time: 1605823708.32
Step 891 of 1500; Error: 633.03794; Time: 1605823710.62
Step 892 of 1500; Error: 633.01357; Time: 1605823713.18
Step 893 of 1500; Error: 632.98925; Time: 160582

Step 1022 of 1500; Error: 630.23317; Time: 1605823987.21
Step 1023 of 1500; Error: 630.21406; Time: 1605823989.03
Step 1024 of 1500; Error: 630.19498; Time: 1605823990.89
Step 1025 of 1500; Error: 630.17593; Time: 1605823992.86
Step 1026 of 1500; Error: 630.15690; Time: 1605823994.70
Step 1027 of 1500; Error: 630.13790; Time: 1605823996.51
Step 1028 of 1500; Error: 630.11893; Time: 1605823998.39
Step 1029 of 1500; Error: 630.09998; Time: 1605824000.07
Step 1030 of 1500; Error: 630.08106; Time: 1605824001.82
Step 1031 of 1500; Error: 630.06216; Time: 1605824003.67
Step 1032 of 1500; Error: 630.04329; Time: 1605824005.36
Step 1033 of 1500; Error: 630.02445; Time: 1605824006.95
Step 1034 of 1500; Error: 630.00563; Time: 1605824008.60
Step 1035 of 1500; Error: 629.98683; Time: 1605824010.26
Step 1036 of 1500; Error: 629.96806; Time: 1605824011.85
Step 1037 of 1500; Error: 629.94932; Time: 1605824013.44
Step 1038 of 1500; Error: 629.93060; Time: 1605824015.00
Step 1039 of 1500; Error: 629.9

Step 1166 of 1500; Error: 627.70666; Time: 1605824303.34
Step 1167 of 1500; Error: 627.69040; Time: 1605824305.31
Step 1168 of 1500; Error: 627.67417; Time: 1605824307.24
Step 1169 of 1500; Error: 627.65795; Time: 1605824309.06
Step 1170 of 1500; Error: 627.64174; Time: 1605824310.96
Step 1171 of 1500; Error: 627.62554; Time: 1605824312.83
Step 1172 of 1500; Error: 627.60937; Time: 1605824314.69
Step 1173 of 1500; Error: 627.59320; Time: 1605824316.62
Step 1174 of 1500; Error: 627.57705; Time: 1605824318.29
Step 1175 of 1500; Error: 627.56091; Time: 1605824320.02
Step 1176 of 1500; Error: 627.54479; Time: 1605824321.76
Step 1177 of 1500; Error: 627.52869; Time: 1605824323.47
Step 1178 of 1500; Error: 627.51259; Time: 1605824325.13
Step 1179 of 1500; Error: 627.49651; Time: 1605824326.98
Step 1180 of 1500; Error: 627.48045; Time: 1605824328.75
Step 1181 of 1500; Error: 627.46440; Time: 1605824330.53
Step 1182 of 1500; Error: 627.44836; Time: 1605824332.25
Step 1183 of 1500; Error: 627.4

Step 1310 of 1500; Error: 625.49561; Time: 1605824562.84
Step 1311 of 1500; Error: 625.48104; Time: 1605824564.62
Step 1312 of 1500; Error: 625.46647; Time: 1605824566.38
Step 1313 of 1500; Error: 625.45192; Time: 1605824568.18
Step 1314 of 1500; Error: 625.43737; Time: 1605824569.96
Step 1315 of 1500; Error: 625.42283; Time: 1605824571.72
Step 1316 of 1500; Error: 625.40831; Time: 1605824573.47
Step 1317 of 1500; Error: 625.39379; Time: 1605824575.23
Step 1318 of 1500; Error: 625.37928; Time: 1605824576.97
Step 1319 of 1500; Error: 625.36478; Time: 1605824578.79
Step 1320 of 1500; Error: 625.35029; Time: 1605824580.54
Step 1321 of 1500; Error: 625.33580; Time: 1605824582.34
Step 1322 of 1500; Error: 625.32133; Time: 1605824584.12
Step 1323 of 1500; Error: 625.30687; Time: 1605824585.87
Step 1324 of 1500; Error: 625.29241; Time: 1605824587.59
Step 1325 of 1500; Error: 625.27796; Time: 1605824589.30
Step 1326 of 1500; Error: 625.26353; Time: 1605824591.05
Step 1327 of 1500; Error: 625.2

Step 1454 of 1500; Error: 623.48122; Time: 1605824804.40
Step 1455 of 1500; Error: 623.46775; Time: 1605824805.92
Step 1456 of 1500; Error: 623.45429; Time: 1605824807.44
Step 1457 of 1500; Error: 623.44084; Time: 1605824808.92
Step 1458 of 1500; Error: 623.42740; Time: 1605824810.41
Step 1459 of 1500; Error: 623.41396; Time: 1605824811.96
Step 1460 of 1500; Error: 623.40053; Time: 1605824813.50
Step 1461 of 1500; Error: 623.38710; Time: 1605824815.01
Step 1462 of 1500; Error: 623.37368; Time: 1605824816.54
Step 1463 of 1500; Error: 623.36027; Time: 1605824818.24
Step 1464 of 1500; Error: 623.34686; Time: 1605824819.76
Step 1465 of 1500; Error: 623.33346; Time: 1605824821.34
Step 1466 of 1500; Error: 623.32006; Time: 1605824822.92
Step 1467 of 1500; Error: 623.30668; Time: 1605824824.51
Step 1468 of 1500; Error: 623.29329; Time: 1605824826.06
Step 1469 of 1500; Error: 623.27992; Time: 1605824827.61
Step 1470 of 1500; Error: 623.26655; Time: 1605824829.16
Step 1471 of 1500; Error: 623.2

##### End Error:
This matrix factorization has 2,500 users instead of only ~900 distinct users. The steps are also reduced from 3000 steps with matrix factorizatiom with disticnt users to 1500. The error converges to around ~622 at step 1500 while the previous example of matrix factorization had converged to an error of ~162 3000. Since the sample size increased, and the number of steps decreased, the increase in error makes sense. This change was mostly due to the program taking very long as the number of users/steps increased. Perhaps in the future, we can attempt to use the whole data set with all the users and items. 

In [68]:
outP = open("rr_p.csv", "w")
outQ = open("rr_q.csv", "w")
np.savetxt(outP, fP, delimiter=',', fmt='%1.4f')
np.savetxt(outQ, fQ, delimiter=',', fmt='%1.4f')

In [69]:
Preds = np.dot(fP,fQ.T)

In [70]:
outPreds = open("jokes_predictions.csv", "w")
np.savetxt(outPreds, Preds, delimiter=',', fmt='%1.4f')

In [71]:
totCount = 0
totError = 0

for u in range(M):
    err_u = 0
    rateCount_u = 0
    for j in range(N):
        if (R1[u,j] > 0): ### Only use known ratings computing error
            rateCount_u += 1
            err_u += abs(np.dot(fP[u],fQ[j]) - R1[u,j])
    print("Mean Absolute Error for User %d = %0.3f" %(u, err_u/rateCount_u))
    totCount += rateCount_u
    totError += err_u
print
print("Overall Mean Absolute Error = %0.3f" %(totError/totCount))

Mean Absolute Error for User 0 = 0.023
Mean Absolute Error for User 1 = 0.024
Mean Absolute Error for User 2 = 0.016
Mean Absolute Error for User 3 = 0.029
Mean Absolute Error for User 4 = 0.028
Mean Absolute Error for User 5 = 0.021
Mean Absolute Error for User 6 = 0.011
Mean Absolute Error for User 7 = 0.011
Mean Absolute Error for User 8 = 0.028
Mean Absolute Error for User 9 = 0.017
Mean Absolute Error for User 10 = 0.024
Mean Absolute Error for User 11 = 0.011
Mean Absolute Error for User 12 = 0.025
Mean Absolute Error for User 13 = 0.035
Mean Absolute Error for User 14 = 0.019
Mean Absolute Error for User 15 = 0.017
Mean Absolute Error for User 16 = 0.002
Mean Absolute Error for User 17 = 0.009
Mean Absolute Error for User 18 = 0.010
Mean Absolute Error for User 19 = 0.013
Mean Absolute Error for User 20 = 0.018
Mean Absolute Error for User 21 = 0.012
Mean Absolute Error for User 22 = 0.019
Mean Absolute Error for User 23 = 0.025
Mean Absolute Error for User 24 = 0.010
Mean Absol

Mean Absolute Error for User 429 = 0.017
Mean Absolute Error for User 430 = 0.015
Mean Absolute Error for User 431 = 0.018
Mean Absolute Error for User 432 = 0.012
Mean Absolute Error for User 433 = 0.021
Mean Absolute Error for User 434 = 0.010
Mean Absolute Error for User 435 = 0.020
Mean Absolute Error for User 436 = 0.014
Mean Absolute Error for User 437 = 0.012
Mean Absolute Error for User 438 = 0.009
Mean Absolute Error for User 439 = 0.012
Mean Absolute Error for User 440 = 0.013
Mean Absolute Error for User 441 = 0.020
Mean Absolute Error for User 442 = 0.025
Mean Absolute Error for User 443 = 0.011
Mean Absolute Error for User 444 = 0.026
Mean Absolute Error for User 445 = 0.011
Mean Absolute Error for User 446 = 0.004
Mean Absolute Error for User 447 = 0.015
Mean Absolute Error for User 448 = 0.025
Mean Absolute Error for User 449 = 0.028
Mean Absolute Error for User 450 = 0.028
Mean Absolute Error for User 451 = 0.019
Mean Absolute Error for User 452 = 0.026
Mean Absolute Er

Mean Absolute Error for User 898 = 0.013
Mean Absolute Error for User 899 = 0.022
Mean Absolute Error for User 900 = 0.007
Mean Absolute Error for User 901 = 0.013
Mean Absolute Error for User 902 = 0.030
Mean Absolute Error for User 903 = 0.014
Mean Absolute Error for User 904 = 0.004
Mean Absolute Error for User 905 = 0.017
Mean Absolute Error for User 906 = 0.009
Mean Absolute Error for User 907 = 0.010
Mean Absolute Error for User 908 = 0.021
Mean Absolute Error for User 909 = 0.019
Mean Absolute Error for User 910 = 0.022
Mean Absolute Error for User 911 = 0.021
Mean Absolute Error for User 912 = 0.020
Mean Absolute Error for User 913 = 0.021
Mean Absolute Error for User 914 = 0.022
Mean Absolute Error for User 915 = 0.095
Mean Absolute Error for User 916 = 0.020
Mean Absolute Error for User 917 = 0.017
Mean Absolute Error for User 918 = 0.020
Mean Absolute Error for User 919 = 0.015
Mean Absolute Error for User 920 = 0.033
Mean Absolute Error for User 921 = 0.010
Mean Absolute Er

Mean Absolute Error for User 1265 = 0.018
Mean Absolute Error for User 1266 = 0.018
Mean Absolute Error for User 1267 = 0.013
Mean Absolute Error for User 1268 = 0.018
Mean Absolute Error for User 1269 = 0.020
Mean Absolute Error for User 1270 = 0.011
Mean Absolute Error for User 1271 = 0.010
Mean Absolute Error for User 1272 = 0.024
Mean Absolute Error for User 1273 = 0.018
Mean Absolute Error for User 1274 = 0.025
Mean Absolute Error for User 1275 = 0.009
Mean Absolute Error for User 1276 = 0.018
Mean Absolute Error for User 1277 = 0.015
Mean Absolute Error for User 1278 = 0.027
Mean Absolute Error for User 1279 = 0.014
Mean Absolute Error for User 1280 = 0.008
Mean Absolute Error for User 1281 = 0.010
Mean Absolute Error for User 1282 = 0.016
Mean Absolute Error for User 1283 = 0.021
Mean Absolute Error for User 1284 = 0.015
Mean Absolute Error for User 1285 = 0.029
Mean Absolute Error for User 1286 = 0.014
Mean Absolute Error for User 1287 = 0.021
Mean Absolute Error for User 1288 

Mean Absolute Error for User 1703 = 0.023
Mean Absolute Error for User 1704 = 0.010
Mean Absolute Error for User 1705 = 0.021
Mean Absolute Error for User 1706 = 0.019
Mean Absolute Error for User 1707 = 0.007
Mean Absolute Error for User 1708 = 0.015
Mean Absolute Error for User 1709 = 0.014
Mean Absolute Error for User 1710 = 0.017
Mean Absolute Error for User 1711 = 0.015
Mean Absolute Error for User 1712 = 0.022
Mean Absolute Error for User 1713 = 0.015
Mean Absolute Error for User 1714 = 0.017
Mean Absolute Error for User 1715 = 0.014
Mean Absolute Error for User 1716 = 0.010
Mean Absolute Error for User 1717 = 0.014
Mean Absolute Error for User 1718 = 0.023
Mean Absolute Error for User 1719 = 0.021
Mean Absolute Error for User 1720 = 0.012
Mean Absolute Error for User 1721 = 0.015
Mean Absolute Error for User 1722 = 0.009
Mean Absolute Error for User 1723 = 0.018
Mean Absolute Error for User 1724 = 0.023
Mean Absolute Error for User 1725 = 0.015
Mean Absolute Error for User 1726 

Mean Absolute Error for User 2136 = 0.021
Mean Absolute Error for User 2137 = 0.021
Mean Absolute Error for User 2138 = 0.013
Mean Absolute Error for User 2139 = 0.016
Mean Absolute Error for User 2140 = 0.012
Mean Absolute Error for User 2141 = 0.027
Mean Absolute Error for User 2142 = 0.011
Mean Absolute Error for User 2143 = 0.010
Mean Absolute Error for User 2144 = 0.017
Mean Absolute Error for User 2145 = 0.011
Mean Absolute Error for User 2146 = 0.011
Mean Absolute Error for User 2147 = 0.014
Mean Absolute Error for User 2148 = 0.020
Mean Absolute Error for User 2149 = 0.024
Mean Absolute Error for User 2150 = 0.013
Mean Absolute Error for User 2151 = 0.016
Mean Absolute Error for User 2152 = 0.012
Mean Absolute Error for User 2153 = 0.011
Mean Absolute Error for User 2154 = 0.012
Mean Absolute Error for User 2155 = 0.025
Mean Absolute Error for User 2156 = 0.011
Mean Absolute Error for User 2157 = 0.016
Mean Absolute Error for User 2158 = 0.026
Mean Absolute Error for User 2159 

##### Overall MAE
The Overall MAE using 2500 users was .018 while the overall MAE of distinct users was only .10. The sample size is larger and thus there is more variance in the system. However, the MAE is still very small and we can hypothesize that the the ratings are fairly accurate. 